In [7]:
# source: https://github.com/sp9103/AXXB-Calibration
import numpy as np
from scipy.spatial.transform import Rotation
from scipy.linalg import expm, logm

In [8]:
def RandomRGen():
    """
    Creates random rotation matrix, SO(3)
    """
    RandR = Rotation.random().as_matrix()
    return RandR

R = RandomRGen()
print(R@R.T) # should be identity

[[ 1.00000000e+00  4.83797787e-17 -9.21361614e-17]
 [ 4.83797787e-17  1.00000000e+00  1.11766430e-16]
 [-9.21361614e-17  1.11766430e-16  1.00000000e+00]]


In [9]:
def RandomTGen():
    """
    Creates random transformation matrix, SE(3)
    """
    RandT = np.zeros((4, 4))
    RandR = RandomRGen()
    t = np.random.rand(3)
    RandT[:3, :3] = RandR
    RandT[:3, 3] = t
    RandT[3, 3] = 1
    return RandT

T = RandomTGen()
print(T)

[[ 0.64867817  0.57802121 -0.49508394  0.63308817]
 [ 0.52261927 -0.8112012  -0.26233892  0.22547292]
 [-0.55325015 -0.08856688 -0.82829354  0.9935673 ]
 [ 0.          0.          0.          1.        ]]


In [10]:
def RandAXXBGen(n=2, noisy=False):
    """
    Creates list of matrices As and Bs to solve AX=XB
    """
    X = RandomTGen()
    A_set = []
    B_set = []
    for i in range(n):
        A = RandomTGen()
        B = np.linalg.inv(X) @ A @ X
        if noisy:
            B += np.random.normal(0, 0.1, size=(4, 4))
        A_set.append(A)
        B_set.append(B)
    return X, A_set, B_set

X, A_set, B_set = RandAXXBGen(n=2, noisy=False)
print(A_set[0] @ X - X @ B_set[0]) # should be all zeros

[[ 5.55111512e-17  0.00000000e+00  0.00000000e+00  1.11022302e-16]
 [-1.38777878e-17  0.00000000e+00 -1.11022302e-16  0.00000000e+00]
 [ 0.00000000e+00  5.55111512e-17 -4.85722573e-17  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00]]


In [29]:
def CalcAXXB(A1, B1, A2, B2, verbose=False):
    """
    Calculate unique (analytical) solution for AX=XB. Assumes no noise.
    Frank C. Park and Bryan J. Martin, "Robot Sensor Calibration: Solving AX=XB on the Euclidean Group", IEEE 1994
    * A. Finding a Unique Solution on SO(3)
    * B. Solution on SE(3)
    """
    # Get rotation matrices and translation vectors
    RotA1 = A1[:3, :3]
    RotB1 = B1[:3, :3]
    RotA2 = A2[:3, :3]
    RotB2 = B2[:3, :3]
    tA1 = A1[:3, 3]
    tB1 = B1[:3, 3]
    tA2 = A2[:3, 3]
    tB2 = B2[:3, 3]
    # Matrix Logarithm mapping
    a1 = np.take(logm(RotA1).reshape(-1), indices=[7, 2, 3])
    b1 = np.take(logm(RotB1).reshape(-1), indices=[7, 2, 3])
    a2 = np.take(logm(RotA2).reshape(-1), indices=[7, 2, 3])
    b2 = np.take(logm(RotB2).reshape(-1), indices=[7, 2, 3])
    # Cross product
    a1xa2 = np.cross(a1, a2)
    b1xb2 = np.cross(b1, b2)
    # Unique solution on SO(3)
    A_ = np.zeros((3, 3))
    B_ = np.zeros((3, 3))
    A_[:, 0] = a1
    A_[:, 1] = a2
    A_[:, 2] = a1xa2
    B_[:, 0] = b1
    B_[:, 1] = b2
    B_[:, 2] = b1xb2
    RotX = A_ @ np.linalg.inv(B_)
    if verbose:
        print("RotX:", RotX)
    # Unique solution on SE(3)
    Left = np.concatenate([RotA1 - np.eye(3), 
                           RotA2 - np.eye(3)], axis=0)
    Right = np.concatenate([RotX @ tB1 - tA1, 
                            RotX @ tB2 - tA2], axis=0)
    tX = np.linalg.pinv(Left) @ Right
    if verbose:
        print("tX:", tX)
    X = np.zeros((4, 4))
    X[:3, :3] = RotX
    X[:3, 3] = tX
    X[3, 3] = 1
    return X


X, A_set, B_set = RandAXXBGen(n=2, noisy=False)
print("True RotX:", X[:3, :3])
print("True tX:", X[:3, 3])
X_pred = CalcAXXB(A_set[0], B_set[0], A_set[1], B_set[1], verbose=True)
print(X_pred - X) # should be all zeros

True RotX: [[ 0.97916564  0.17705449  0.0994302 ]
 [-0.19435486  0.95899514  0.20628746]
 [-0.05882896 -0.22131434  0.97342648]]
True tX: [0.28464389 0.04607183 0.58713073]
RotX: [[ 0.97916564  0.17705449  0.0994302 ]
 [-0.19435486  0.95899514  0.20628746]
 [-0.05882896 -0.22131434  0.97342648]]
tX: [0.28464389 0.04607183 0.58713073]
[[ 7.77156117e-16 -3.13638004e-15 -2.35922393e-16  1.66533454e-16]
 [-8.32667268e-17 -6.66133815e-16 -1.22124533e-15 -2.84494650e-16]
 [-2.63677968e-16  2.08166817e-15 -1.11022302e-16 -3.33066907e-16]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00]]


In [42]:
def NormalizeR(R):
    """
    Normalizes rotation matrix for noisy matrices which do not satisfy properties of SO(3)
    *** Need to find a better way to normalize... Sometimes produces error as matrix logarithm gives complex numbers
    """
    assert R.shape == (3, 3)
    for i in range(3):
        v = R[:, i]
        norm = np.linalg.norm(v)
        R[:, i] = v / norm
    return R

def LeastSquaresAXXB(A_set, B_set, verbose=False):
    """
    Calculate least-squares solution for AX=XB with noise.
    Frank C. Park and Bryan J. Martin, "Robot Sensor Calibration: Solving AX=XB on the Euclidean Group", IEEE 1994
    * IV. A Least-Squares Solution
    """
    assert len(A_set) == len(B_set)
    n = len(A_set)
    M = np.zeros((3, 3))
    tA_set = []
    tB_set = []
    RotA_set = []
    for i in range(n):
        Ai = A_set[i]
        Bi = B_set[i]
        # Get rotation matrices and translation vectors
        RotAi = NormalizeR(Ai[:3, :3])
        RotBi = NormalizeR(Bi[:3, :3])
        tAi = Ai[:3, 3]
        tBi = Bi[:3, 3]
        # Matrix Logarithm mapping
        ai = np.take(logm(RotAi).reshape(-1), indices=[7, 2, 3])
        bi = np.take(logm(RotBi).reshape(-1), indices=[7, 2, 3])
        # Add onto matrix M
        M += bi.reshape(3, 1) @ ai.reshape(1, 3)
        # Append
        tA_set.append(tAi)
        tB_set.append(tBi)
        RotA_set.append(RotAi)
    # Solution on SO(3)
    MTM = M.T @ M
    w, v = np.linalg.eig(MTM)
    w_ = np.power(w, -0.5)
    Diag = np.diag(w_)
    RotX = v @ Diag @ np.linalg.inv(v) @ M.T
    RotX = NormalizeR(RotX)
    if verbose:
        print("RotX:", RotX)
    # Solution on SE(3)
    C_set = []
    d_set = []
    for i in range(n):
        Ci = np.eye(3) - RotA_set[i]
        di = tA_set[i].reshape(3, 1) - RotX @ tB_set[i].reshape(3, 1)
        C_set.append(Ci)
        d_set.append(di)
    C = np.concatenate(C_set, axis=0)
    d = np.concatenate(d_set, axis=0)
    tX = (np.linalg.pinv(C) @ d).reshape(-1)
    if verbose:
        print("tX:", tX)
    X = np.zeros((4, 4))
    X[:3, :3] = RotX
    X[:3, 3] = tX
    X[3, 3] = 1
    return X

X, A_set, B_set = RandAXXBGen(n=10, noisy=True)
print("True RotX:", X[:3, :3])
print("True tX:", X[:3, 3])
X_pred = LeastSquaresAXXB(A_set, B_set, verbose=True)
print(X_pred - X) # goal: all zeros (***noisy)

True RotX: [[-0.88987968  0.25099431  0.38094095]
 [-0.24930705  0.43175128 -0.86685456]
 [-0.3820473  -0.86636753 -0.32163204]]
True tX: [0.30055905 0.79560573 0.60009298]
RotX: [[-0.87060887  0.34610231  0.34964753]
 [-0.27971605  0.23643076 -0.93051568]
 [-0.40472106 -0.90791722 -0.10902834]]
tX: [0.31435148 0.81467859 0.54420694]
[[ 0.01927082  0.095108   -0.03129342  0.01379243]
 [-0.030409   -0.19532052 -0.06366112  0.01907286]
 [-0.02267375 -0.0415497   0.2126037  -0.05588604]
 [ 0.          0.          0.          0.        ]]
